In [1]:
from pathlib import Path
import pandas as pd

In [2]:
# Базовые пути
BASE = Path("..").resolve()
RAW_BASE = BASE / "data" / "raw" / "fbref"

SEASONS = ["epl_2019-2020","epl_2020-2021","epl_2021-2022","epl_2022-2023","epl_2023-2024","epl_2024-2025"]
FILES = ["schedule_results.csv", "standings.csv", "team_standard_stats.csv"]

In [3]:
# Быстрая проверка наличия файлов по каждому сезону
summary = []
for s in SEASONS:
    d = RAW_BASE / s
    row = {"season": s}
    for f in FILES:
        p = d / f
        row[f] = "OK" if p.exists() else "MISSING"
        if p.exists():
            try:
                df = pd.read_csv(p, nrows=2)
                row[f + "_cols"] = df.shape[1]
            except Exception as e:
                row[f] = f"READ_ERR: {e}"
    summary.append(row)

pd.DataFrame(summary)

,season,schedule_results.csv,schedule_results.csv_cols,standings.csv,standings.csv_cols,team_standard_stats.csv,team_standard_stats.csv_cols
0,epl_2019-2020,OK,14,OK,19,OK,32
1,epl_2020-2021,OK,14,OK,19,OK,32
2,epl_2021-2022,OK,14,OK,19,OK,32
3,epl_2022-2023,OK,14,OK,19,OK,32
4,epl_2023-2024,OK,14,OK,19,OK,32
5,epl_2024-2025,OK,14,OK,19,OK,32


In [4]:
all_standings = []
all_teamstats = []

for s in SEASONS:
    d = RAW_BASE / s
    try:
        st = pd.read_csv(d / "standings.csv")
        st["season"] = s
        all_standings.append(st)
    except Exception as e:
        print(f"[ERR] standings {s}: {e}")

    try:
        ts = pd.read_csv(d / "team_standard_stats.csv")
        ts["season"] = s
        all_teamstats.append(ts)
    except Exception as e:
        print(f"[ERR] team stats {s}: {e}")

standings_all = pd.concat(all_standings, ignore_index=True)
teamstats_all = pd.concat(all_teamstats, ignore_index=True)

print("✅ Объединение завершено!")
print("Standings shape:", standings_all.shape)
print("Team stats shape:", teamstats_all.shape)

✅ Объединение завершено!
Standings shape: (120, 20)
Team stats shape: (120, 33)


In [5]:
import pandas as pd

# Проверим первые строки и столбцы для одного сезона (например, 2022-2023)
season_check = "epl_2022-2023"
check_path = RAW_BASE / season_check

# Загружаем фикстуры
fx = pd.read_csv(check_path / "schedule_results.csv")

print(f"📅 {season_check} — schedule_results.csv")
print(f"Размер: {fx.shape}")
print("Столбцы:", list(fx.columns))
display(fx.head(3))

📅 epl_2022-2023 — schedule_results.csv
Размер: (392, 14)
Столбцы: ['wk', 'day', 'date', 'time', 'home', 'xg', 'score', 'xg_1', 'away', 'attendance', 'venue', 'referee', 'match_report', 'notes']


,wk,day,date,time,home,xg,score,xg_1,away,attendance,venue,referee,match_report,notes
0,1,Fri,2022-08-05,20:00 (21:00),Crystal Palace,1.2,0–2,1.0,Arsenal,25286,Selhurst Park,Anthony Taylor,Match Report,NaN
1,1,Sat,2022-08-06,12:30 (13:30),Fulham,1.2,2–2,1.2,Liverpool,22207,Craven Cottage,Andy Madley,Match Report,NaN
2,1,Sat,2022-08-06,15:00 (16:00),Newcastle Utd,1.7,2–0,0.3,Nott'ham Forest,52245,St James' Park,Simon Hooper,Match Report,NaN


In [6]:
from collections import Counter

cols_per_season = {}
for s in SEASONS:
    path = RAW_BASE / s / "schedule_results.csv"
    df = pd.read_csv(path)
    cols_per_season[s] = list(df.columns)

# Проверим уникальные наборы колонок
unique_structures = Counter(tuple(cols) for cols in cols_per_season.values())
print(f"Уникальных структур: {len(unique_structures)}")

# Если всё ок, выведет 1
if len(unique_structures) == 1:
    print("✅ Все сезоны имеют одинаковую структуру fixtures.csv")
else:
    print("⚠️ Различия найдены — нужно проверить вручную")
    for k, v in cols_per_season.items():
        print(f"{k}: {v}")

Уникальных структур: 1
✅ Все сезоны имеют одинаковую структуру fixtures.csv


In [7]:
import re
import numpy as np
import pandas as pd
from pathlib import Path

# ожидается, что SEASONS и RAW_BASE уже определены в ноутбуке
PROCESSED_DIR = (RAW_BASE.parent.parent / "processed" / "fbref")
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
out_path = PROCESSED_DIR / "epl_fixtures_all.csv"

def parse_score_to_cols(series):
    """Из '2–1' / '2-1' -> два столбца (home_goals, away_goals)."""
    def f(s):
        if pd.isna(s):
            return np.nan, np.nan
        m = re.search(r"(\d+)\s*[–-]\s*(\d+)", str(s))
        return (int(m.group(1)), int(m.group(2))) if m else (np.nan, np.nan)
    return series.apply(lambda s: pd.Series(f(s), index=["home_goals","away_goals"]))

frames = []
for s in SEASONS:
    df = pd.read_csv(RAW_BASE / s / "schedule_results.csv")
    df["season"] = s
    # унифицируем имена на всякий случай
    df.columns = (
        df.columns.str.strip().str.lower()
          .str.replace(" ", "_").str.replace("%","pct")
          .str.replace("/", "_").str.replace("-", "_", regex=False)
    )
    # распарсим счёт
    if "score" in df.columns:
        parsed = parse_score_to_cols(df["score"])
        df = pd.concat([df, parsed], axis=1)
    frames.append(df)

fixtures_all = pd.concat(frames, ignore_index=True)

# оставим только сыгранные матчи (есть голы у обеих сторон)
fixtures_all = fixtures_all.dropna(subset=["home_goals","away_goals"]).copy()

# аккуратно приведём типы
for col in ["home_goals","away_goals"]:
    fixtures_all[col] = fixtures_all[col].astype(int)

# порядок столбцов (если есть)
cols_order = [c for c in ["season","wk","date","time","home","away","score","home_goals","away_goals","xg","xg_1","venue","referee","attendance"] if c in fixtures_all.columns]
fixtures_all = fixtures_all[[*cols_order, *[c for c in fixtures_all.columns if c not in cols_order]]]

# сохранение
fixtures_all.to_csv(out_path, index=False, encoding="utf-8")

print(f"✅ Объединено фикстур: {fixtures_all.shape[0]} строк, {fixtures_all.shape[1]} столбцов")
print(f"📁 Сохранено в: {out_path}")
fixtures_all.head(3)

✅ Объединено фикстур: 2280 строк, 17 столбцов
📁 Сохранено в: /Users/kekc/Projects/GIT/sports-stats-analysis/data/processed/fbref/epl_fixtures_all.csv


,season,wk,date,time,home,away,score,home_goals,away_goals,xg,xg_1,venue,referee,attendance,day,match_report,notes
0,epl_2019-2020,1,2019-08-09,20:00 (21:00),Liverpool,Norwich City,4–1,4,1,1.8,0.9,Anfield,Michael Oliver,53333,Fri,Match Report,NaN
1,epl_2019-2020,1,2019-08-10,12:30 (13:30),West Ham,Manchester City,0–5,0,5,1.1,3.2,London Stadium,Mike Dean,59870,Sat,Match Report,NaN
2,epl_2019-2020,1,2019-08-10,15:00 (16:00),Bournemouth,Sheffield Utd,1–1,1,1,1.3,1.3,Vitality Stadium,Kevin Friend,10714,Sat,Match Report,NaN
